In [ ]:
import sagemaker
from sagemaker import Model
from sagemaker.serverless import ServerlessInferenceConfig

In [78]:
import json
import sagemaker
ECR_IMAGE = "637423382292.dkr.ecr.us-west-2.amazonaws.com/llm_test:latest"

sm = sagemaker.Session().sagemaker_runtime_client
sagemaker_role = sagemaker.get_execution_role()

In [ ]:
BUCKET_NAME = 'workhsop-llama-weights'
model_name = 'debug-v2'
model = sagemaker.model.Model(
        name=model_name, 
        image_uri='637423382292.dkr.ecr.us-west-2.amazonaws.com/llm_test:latest',
        role=sagemaker_role,
        env={'BUCKET_NAME': BUCKET_NAME, 'LORA_FILENAME': 'lora_tensors.safetensors'}
    )
model.create()

endpoint = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge', endpoint_name=model_name, volume_size=50, container_startup_health_check_timeout=600)

Using already existing model: debug-v2


----

In [136]:
response = sm.invoke_endpoint(
    EndpointName=model_name,
    Body=json.dumps({'query': 'show me fire scars in maui from august 13, 2023', 'content': 'hello world'}).encode(encoding="UTF-8"),
    ContentType="application/json"
)

response['Body'].read()

b'{"models":"\\n{\\n  \\"datetime\\": \\"2024-05-30T00:00:00+00:00\\",\\n  \\"location\\": \\"London, UK\\"\\n}"}'

In [70]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [137]:
model.delete_model()
sess.delete_endpoint_config(model_name)
sess.delete_endpoint(model_name)